In [ ]:
# Load the data from its source.
df=spark.read.load(“/databricks/DP203/learning/books/book.delta")
# Write the data and save the table name as book.
table_name = “book”
df.write.saveAsTable(table_name)


In [ ]:
%sql

-- Upserting to a table.

CREATE OR REPLACE TEMP VIEW book_updates AS
VALUES
  (998, 'Into the World', 'John', 'Science Fiction', 9.99, 4.5),
  (999, 'First Step', 'Adams', 'Romance', 7.99, 4.2),
  (1000, 'Great War', 'Stephen', 'History', 19.99, 4.8),
  (2001, 'Dream World', 'Saliken', 'Fantasy', 8.99, 4.4),
  (2002, 'little boy in town', 'Suren', 'Science Fiction', 9.99, 4.9),
  (2003, 'Hope you can', 'David', 'New-age', 14.99, 4.6)
AS book_updates(id, title, author, genre, price, rating);

MERGE INTO book
USING book_updates
ON book.id = book_updates.id
WHEN MATCHED THEN 
  UPDATE SET 
    title = book_updates.title,
    author = book_updates.author,
    genre = book_updates.genre,
    price = book_updates.price,
    rating = book_updates.rating
WHEN NOT MATCHED THEN 
  INSERT (id, title, author, genre, price, rating) 
  VALUES (book_updates.id, book_updates.title, book_updates.author, book_updates.genre, book_updates.price, book_updates.rating);

In [ ]:
# Reading a book Table
book_df = spark.read.table(table_name)

display(book_df)

In [ ]:
# Writing to a table To append
book_df.write.mode("append").saveAsTable("book")

# To overwrite
book_df.write.mode("overwrite").saveAsTable("book")

In [ ]:
# Using Optimze command to merge the small files

from delta.tables import *

# Enable bin packing (compaction) for a Delta Lake table
book_table = DeltaTable.forPath(spark, "/path-to-delta-table")
book_table.optimize().executeCompaction()
